# Reduced Rank Model

In [121]:
import numpy as np
import os
import torch
from models.reduced_rank_model import ReducedRankModel

DATA_PATH = 'raw_data/full_data/'

TOTAL_RECORDINGS = 3
RECORDING_IDS = [5, 6, 8]
TRAIN_SPLIT = 0.8
TRAINING_ITERATIONS = 1001
RANK = 3

In [122]:
files = os.listdir(DATA_PATH)
X_files = [f for f in files if f.endswith('.npy') and f.startswith('X')]
X_files.sort()
Y_files = [f for f in files if f.endswith('.npy') and f.startswith('Y')]
Y_files.sort()

X_files = np.array(X_files)
Y_files = np.array(Y_files)

X_list = [np.load(DATA_PATH + f) for f in X_files[RECORDING_IDS]]
Y_list = [np.load(DATA_PATH + f) for f in Y_files[RECORDING_IDS]]

In [123]:
print(len(X_list))
print(X_list[0].shape)

3
(605, 172, 40)


In [124]:
X_train = [X[:int(TRAIN_SPLIT * X.shape[0])] for X in X_list]
X_test = [X[int(TRAIN_SPLIT * X.shape[0]):] for X in X_list]
Y_train = [Y[:int(TRAIN_SPLIT * Y.shape[0])] for Y in Y_list]
Y_test = [Y[int(TRAIN_SPLIT * Y.shape[0]):] for Y in Y_list]

# Shuffle within each recording
for i in range(TOTAL_RECORDINGS):
    indices = np.arange(X_train[i].shape[0])
    np.random.shuffle(indices)
    X_train[i] = X_train[i][indices]
    Y_train[i] = Y_train[i][indices]

In [125]:
print(len(X_train))
print(X_train[0].shape)

3
(484, 172, 40)


# Training

In [126]:
def train_step(recording, X_train_recording, Y_train_recording, model, optimizer, loss_fn):
    optimizer.zero_grad()
    
    X_train_recording = torch.from_numpy(X_train_recording).float()
    Y_train_recording = torch.from_numpy(Y_train_recording).float()

    Y_pred = model(recording, X_train_recording)
    
    loss = loss_fn(Y_pred, Y_train_recording)
    loss.backward()
    optimizer.step()
    
    return loss.item()

In [127]:
reduced_rank_model = ReducedRankModel(
    TOTAL_RECORDINGS,
    [X_train[i].shape[1] for i in range(TOTAL_RECORDINGS)], 
    40, 
    RANK
)

optimizer = torch.optim.Adam(reduced_rank_model.parameters(), lr=0.001)
loss_fn = torch.nn.MSELoss()

loss = []

for training_iteration in range(TRAINING_ITERATIONS):
    # Get a random permutation of numbers from 0 to TOTAL_RECORDINGS
    # session_order = np.arange(TOTAL_RECORDINGS)
    # np.random.shuffle(session_order)

    loss_iter = [recordings for recordings in range(TOTAL_RECORDINGS)]

    for recording in range(TOTAL_RECORDINGS):
        loss_item = train_step(recording, X_train[recording], Y_train[recording], reduced_rank_model, optimizer, loss_fn)
        loss_iter[recording] = loss_item
        if training_iteration % 100 == 0:
            print(f'Iteration {training_iteration}, Recording {recording}, Loss {loss_item}')
    
    loss.append(loss_iter)

Iteration 0, Recording 0, Loss 12.01142692565918
Iteration 0, Recording 1, Loss 9.93109130859375
Iteration 0, Recording 2, Loss 1.8140441179275513
Iteration 100, Recording 0, Loss 2.1077847480773926
Iteration 100, Recording 1, Loss 2.1138219833374023
Iteration 100, Recording 2, Loss 1.5254333019256592
Iteration 200, Recording 0, Loss 1.2435201406478882
Iteration 200, Recording 1, Loss 0.9489120841026306
Iteration 200, Recording 2, Loss 1.1412339210510254
Iteration 300, Recording 0, Loss 0.930820107460022
Iteration 300, Recording 1, Loss 0.6672452688217163
Iteration 300, Recording 2, Loss 0.8821995854377747
Iteration 400, Recording 0, Loss 0.7663208842277527
Iteration 400, Recording 1, Loss 0.5310424566268921
Iteration 400, Recording 2, Loss 0.7134352326393127
Iteration 500, Recording 0, Loss 0.6589599847793579
Iteration 500, Recording 1, Loss 0.44114044308662415
Iteration 500, Recording 2, Loss 0.6019469499588013
Iteration 600, Recording 0, Loss 0.5852888822555542
Iteration 600, Record

In [128]:
# Plot the loss curve using plotly graphical objects
import plotly.graph_objects as go

loss = np.array(loss)

fig = go.Figure()
for recording in range(TOTAL_RECORDINGS):
    fig.add_trace(go.Scatter(y=loss[:, recording], name=f'Recording {recording}'))
fig.update_layout(title=f'Loss Curve Rank {RANK}', xaxis_title='Iteration', yaxis_title='Loss')
fig.show()

# Evaluation

In [129]:
def evaluate_recording(recording, X_test_recording, Y_test_recording, model, optimizer, loss_fn, plot=False):
    # Test the model
    model.eval()
    
    X_test_recording = torch.from_numpy(X_test_recording).float()
    Y_test_recording = torch.from_numpy(Y_test_recording).float()
    Y_pred = model(recording, X_test_recording)
    loss = loss_fn(Y_pred, Y_test_recording)

    if plot:
        for t in range(3):
            fig = go.Figure()
            fig.add_trace(go.Scatter(y=Y_test_recording[t], name='Actual'))
            fig.add_trace(go.Scatter(y=Y_pred[t].detach().numpy(), name='Predicted'))
            fig.update_layout(title=f'Actual and Predicted Wheel Speeds, Recording {recording}, Trial {t}', xaxis_title='Time', yaxis_title='Value')
            fig.show()

    return loss.item()

In [130]:
# Print all of the model parameters
# for param in reduced_rank_model.parameters():
#     print(param.data.shape)
#     print(param.data)

print(f"=== U matrices, length: {len(reduced_rank_model.Us)} ===")
print(reduced_rank_model.Us)
print(f"=== V matrices, shape: {reduced_rank_model.V.shape} ===")
print(reduced_rank_model.V)
# print(f"=== bias, shape: {reduced_rank_model.bias.shape} ===")
# print(reduced_rank_model.bias)

# print(reduced_rank_model.parameters())


=== U matrices, length: 3 ===
[Parameter containing:
tensor([[ 0.6042,  0.9310, -0.6430],
        [-0.1055,  1.5804,  0.5292],
        [ 1.3383,  1.1797, -0.1868],
        [-0.2249,  1.5354,  0.0866],
        [ 0.5325, -1.5157, -0.0638],
        [ 0.6682,  0.7028, -0.8300],
        [-1.1893,  2.0864,  0.6117],
        [ 0.0032,  1.1129,  1.2300],
        [-0.7916,  0.0919,  0.1521],
        [-0.9817,  0.6601,  1.1497],
        [ 1.7015, -0.2099,  0.1676],
        [-0.2629, -0.3463, -1.6124],
        [-1.0448,  0.7643,  0.5226],
        [-1.4451,  1.0123, -0.2704],
        [ 0.4192, -0.6872,  1.4446],
        [-0.1927, -0.8649, -0.5248],
        [ 1.1718, -0.5800, -0.6844],
        [ 0.3528,  0.6477,  1.0064],
        [-0.4612, -0.3097,  0.5244],
        [-1.3825, -0.3373,  0.1173],
        [-2.0625,  1.1321,  0.2220],
        [-0.7814,  1.2736,  0.1770],
        [ 1.2369,  0.0464, -1.7374],
        [ 2.3847,  2.3731, -0.8053],
        [ 1.5561, -0.2946, -1.7265],
        [-0.6937,  0.0

In [131]:
for recording in range(TOTAL_RECORDINGS):
    loss_item = evaluate_recording(recording, X_test[recording], Y_test[recording], reduced_rank_model, optimizer, loss_fn, plot=True)
    print(f'Evaluation, Recording {recording}, Loss {loss_item}')

Evaluation, Recording 0, Loss 0.5956053733825684


Evaluation, Recording 1, Loss 0.3791147470474243


Evaluation, Recording 2, Loss 0.4773322343826294
